In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
def model_eval(model, data_loader, return_preds = False):
  model.eval()
  y_preds = []
  y_true = []
  with torch.no_grad():
    for inputs, labels in data_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      y_preds.append(outputs)
      y_true.append(labels)

  y_preds = torch.cat(y_preds).cpu().detach().numpy()
  y_true = torch.cat(y_true).cpu().detach().numpy()

  if return_preds:
    return {'y_true': y_true, 'y_preds': y_preds}
  else:
    return roc_auc_score(y_true, y_preds)

In [ ]:
################################
#### Clean and Process Data ####
################################

data = 'TRs_annotated_cleaned.csv'
df_full = pd.read_csv(data)

col_names = df_full.columns
categorical = [var for var in df_full.columns if df_full[var].dtype=='O']
numerical = [var for var in df_full.columns if df_full[var].dtype!='O']

ids = df_full.id

df = df_full.drop(['id', 'location', 'region', 'tissue_simple'], axis=1)

one_hot_gene_type = pd.get_dummies(df.gene_type, prefix="gene_type", drop_first=True, dtype=int)
new_df = pd.concat([df[numerical], one_hot_gene_type], axis=1)
new_df.drop(['gene_type_intergenic'], axis=1, inplace=True)

In [ ]:
test_chr = "chr13_" # chr1, chr2, chr13

test_indices = np.array([index for index, string in enumerate(ids) if test_chr in string])
test_ids = df_full.iloc[new_df.index.isin(test_indices)].id

X_train = new_df.iloc[~new_df.index.isin(test_indices)]
X_test = new_df.iloc[new_df.index.isin(test_indices)]

y_train = new_df['label'].iloc[~new_df.index.isin(test_indices)]
y_test= new_df['label'].iloc[new_df.index.isin(test_indices)]

X_train = X_train.drop('label', axis=1)
X_test= X_test.drop('label', axis=1)

print('Size of Training Set:', X_train.shape, 'Shape of Training Labels:', y_train.shape)
print('Size of Testing Set:', X_test.shape, 'Shape of Testing Labels:', y_test.shape)

Size of Training Set: (35887, 56) Shape of Training Labels: (35887,)
Size of Testing Set: (1155, 56) Shape of Testing Labels: (1155,)


In [ ]:
# List of column names
all_columns = X_train.columns

# List to store column names where all values are 0 or 1
binary_columns = []

# Iterate over each column
for column in all_columns:
    unique_values = X_train[column].unique()
    # Check if unique values contain only 0 and 1
    if set(unique_values) == {0, 1}:
        binary_columns.append(column)

numeric_columns = [elem for elem in all_columns if elem not in binary_columns]

# Create a ColumnTransformer to apply different transformations to numeric and binary columns
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', MinMaxScaler(), numeric_columns),
        ('binary', 'passthrough', binary_columns)
    ])

# Define the pipeline with the preprocessing steps
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the training data
X_train_processed = pipeline.fit_transform(X_train)

# Transform the test data using the fitted pipeline
X_test_processed = pipeline.transform(X_test)

# Convert the processed data back to DataFrame (optional)
X_train_processed = pd.DataFrame(X_train_processed, columns=numeric_columns + binary_columns)
X_test_processed = pd.DataFrame(X_test_processed, columns=numeric_columns + binary_columns)

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_processed.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_processed.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create a PyTorch dataset and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Find outcome rates
ys = []
for inputs, labels in train_loader:
  ys.append(labels)

ys = torch.cat(ys)
print('Outcome rate in Training:', torch.mean(ys).detach().numpy().round(3))

ys = []
for inputs, labels in test_loader:
  ys.append(labels)

ys = torch.cat(ys)
print('Outcome rate in Testing:', torch.mean(ys).detach().numpy().round(3))

Outcome rate in Training: 0.152
Outcome rate in Testing: 0.148


In [ ]:
class MLP(nn.Module):
    def __init__(self, dropout):
        super(MLP, self).__init__()

        self.dropout = nn.Dropout(p=dropout)

        self.fc1 = nn.LazyLinear(500, bias=False)
        self.bn1 = nn.BatchNorm1d(500)

        self.fc2 = nn.Linear(500, 1000, bias=False)
        self.bn2 = nn.BatchNorm1d(1000)

        self.fc3 = nn.Linear(1000, 500, bias=False)
        self.bn3 = nn.BatchNorm1d(500)

        self.fc4 = nn.Linear(500, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout(x)

        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = self.dropout(x)

        x = F.sigmoid(self.fc4(x))
        return x.squeeze(1)


In [ ]:
torch.manual_seed(1)
model = MLP(0.5)
model

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


MLP(
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): LazyLinear(in_features=0, out_features=500, bias=False)
  (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=500, out_features=1000, bias=False)
  (bn2): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=1000, out_features=500, bias=False)
  (bn3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=500, out_features=1, bias=True)
)

In [ ]:
epoch_val_roc = []
epoch_train_roc = []
best_auc = 0
counter = 0
EARLY_STOP = np.inf

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define loss function and optimizer
### OOPS! Just realized nn.CrossEntropyLoss() expects raw logits not the predicted probabilities will rerun later to see if results change
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.to(device)

# Train the model
num_epochs = 1000
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    train_auc = model_eval(model, train_loader)

    epoch_train_roc.append(train_auc)

100%|██████████| 1000/1000 [42:05<00:00,  2.53s/it]


In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/BMI 776 Final Project/Results/Models/Neural Network/LOOC/model_chr13.pth')

In [ ]:
outputs = model_eval(model, test_loader, return_preds=True)

In [ ]:
pd.DataFrame(outputs, index=test_ids).sort_values('y_preds', ascending=False).to_csv('/content/drive/My Drive/BMI 776 Final Project/Results/Models/Neural Network/LOOC/chr13_scores.csv')